In [1]:
from modules.dataset import split_subject_data, read_json_file

In [2]:
json = read_json_file("processed_data/dataset_index_mapping.json")

test, train = split_subject_data(json, 40, 42)

In [3]:
print(len(test))
print(len(train))

7882
966


In [4]:
print(test)

[('181', 0), ('181', 1), ('181', 2), ('181', 3), ('181', 4), ('181', 5), ('181', 6), ('181', 7), ('181', 8), ('181', 9), ('181', 10), ('181', 11), ('181', 12), ('181', 13), ('181', 14), ('181', 15), ('181', 16), ('181', 17), ('181', 18), ('181', 19), ('181', 20), ('181', 21), ('181', 22), ('181', 23), ('181', 24), ('181', 25), ('031', 0), ('031', 1), ('031', 2), ('031', 3), ('031', 4), ('031', 5), ('031', 6), ('031', 7), ('031', 8), ('031', 9), ('031', 10), ('031', 11), ('031', 12), ('031', 13), ('031', 14), ('031', 15), ('031', 16), ('031', 17), ('031', 18), ('031', 19), ('031', 20), ('031', 21), ('306', 0), ('306', 1), ('306', 2), ('306', 3), ('306', 4), ('306', 5), ('306', 6), ('306', 7), ('306', 8), ('306', 9), ('306', 10), ('306', 11), ('306', 12), ('306', 13), ('306', 14), ('306', 15), ('306', 16), ('306', 17), ('306', 18), ('306', 19), ('306', 20), ('306', 21), ('306', 22), ('306', 23), ('109', 0), ('109', 1), ('109', 2), ('109', 3), ('109', 4), ('109', 5), ('109', 6), ('109', 7